# Conhecendo os dados

### Desafio: utilizando parâmetros do json_normalize

Ao normalizar o objeto JSON durante a aula passamos simplesmente ```pd.json_normalize(<dados>)``` mas o método json_normalize possui diversos parâmetros para trabalhar com estruturas de dados mais complexas.

Você recebeu a tarefa de normalizar o seguinte arquivo JSON chamado ```“informacoes.json”``` referente a algumas informações de identificação de um cliente. O arquivo possui o seguinte conteúdo:

```python
{
  "nome": "João",
  "idade": 28,
  "enderecos": [
    {
      "tipo": "casa",
      "rua": "Rua A",
      "numero": 123,
      "cidade": "São Paulo"
    },
    {
      "tipo": "trabalho",
      "rua": "Rua B",
      "numero": 456,
      "cidade": "Rio de Janeiro"
    }
  ]
}
```

O resultado final deve ser um Pandas DataFrame que possui a seguinte estrutura:

![tabela](assets/pandas_limpeza_tratamento_de_dados_aula1.png)

Como podemos fazer isso?

In [ ]:
import pandas as pd
import json

In [ ]:
url = 'pandas-dados/informacoes.json'

In [ ]:
dados = pd.read_json(url)
dados.head()

In [ ]:
pd.json_normalize(dados['enderecos'])

In [ ]:
with open(url) as arquivo:
    json_bruto = json.load(arquivo)

In [ ]:
json_bruto

In [ ]:
dados_normlizados = pd.json_normalize(json_bruto, record_path='enderecos',meta=["nome", "idade"])
dados_normlizados.head()

### Desafio: tratando os dados

Você recebeu e fez o download o arquivo JSON chamado “dados.json” que possui o seguinte conteúdo:

```json
{
  "pessoas": [
    {
      "nome": "João",
      "idade": "25",
      "endereco": {
        "rua": "Rua A",
        "numero": 123,
        "cidade": "São Paulo"
      },
      "telefones": [
        "11 1111-1111",
        "11 2222-2222"
      ]
    },
    {
      "nome": "Maria",
      "idade": 30,
      "endereco": {
        "rua": "",
        "numero": 456,
        "cidade": "Rio de Janeiro"
      },
      "telefones": [
        "21 3333-3333"
      ]
    }
  ]
}
```

Sua tarefa é realizar alguns tratamentos e deixar esse arquivo JSON como um Pandas DataFrame, sem dados vazios e com a tipagem das colunas de forma correta. Como isso pode ser feito?

In [ ]:
import pandas as pd
import json

In [ ]:
url = 'pandas-dados/dados.json'

In [ ]:
with open(url) as arquivo:
    dados = json.load(arquivo)

In [ ]:
dados

In [ ]:
df = pd.json_normalize(dados, record_path=['pessoas'])
df

In [ ]:
df.info()

analisando as informacoes acima, preciso corrigir:

1. acertar o tipo da coluna idade
2. retirar a linha com dados nulos

In [ ]:
df['idade'] = df['idade'].astype(int)

In [ ]:
df.info()

In [ ]:
filtro_nulas = df['endereco.rua'] != ''

In [ ]:
df_filtrado = df[filtro_nulas].copy()
df_filtrado

### Desafio: tratando uma base de dados

Para facilitar a análise dos dados de cadastro de cursos de uma plataforma, você recebeu um arquivo chamado [cursos_cadastrados.json](https://cdn3.gnarususercontent.com.br/2929-pandas/cursos_cadastrados.json). Você foi informado que esses dados podem apresentar problemas que podem prejudicar a análise e interpretação correta dos dados.

Para resolver isso, foi solicitado que você identifique e remova as amostras que apresentam os seguintes problemas:

- Valores nulos: são valores ausentes em algumas das colunas que devem ser preenchidos para que a análise seja correta.
- Duplicatas: registros iguais que podem prejudicar a análise dos dados, já que estão representando a mesma informação.
- Strings vazias: valores de texto que não apresentam nenhum conteúdo escrito, o que pode dificultar a análise de dados, principalmente se a coluna tiver muitos valores assim.
- Conversão de tipos: algumas colunas do arquivo podem estar no tipo de dados errado, como texto em vez de número, o que pode prejudicar a análise. Nesse caso você deve realizar a conversão para o tipo correto de cada coluna.

Dentro desse contexto, você precisará de uma abordagem sistemática para limpar os dados. Assim, como isso pode ser feito?

Lembrando que o conteúdo de cursos_cadastrados.json é:

```python 
[
    {
        "curso": "Introdução à programação",
        "categoria": "Programação",
        "carga_horaria": "20 horas",
        "concluintes": 100,
        "data_inicio": "2022-01-01",
        "data_conclusao": "2022-01-20",
        "descricao": "Curso introdutório à programação com Python",
        "preco": "99.90",
        "instrutor": {
            "nome": "João Silva",
            "email": "joao.silva@emailaleatorio.com",
            "telefone": "(11) 9999-9999"
        }
    },
    {
        "curso": "Excel para iniciantes",
        "categoria": "Produtividade",
        "carga_horaria": null,
        "concluintes": null,
        "data_inicio": null,
        "data_conclusao": null,
        "descricao": null,
        "preco": null,
        "instrutor": {
            "nome": "Maria Oliveira",
            "email": "maria.oliveira@emailaleatorio.com",
            "telefone": "(11) 8888-8888"
        }
    },
    {
        "curso": "Marketing digital para negócios",
        "categoria": "Marketing",
        "carga_horaria": "30 horas",
        "concluintes": 75,
        "data_inicio": "2022-03-01",
        "data_conclusao": "2022-03-31",
        "descricao": "Curso introdutório em marketing",
        "preco": 89.90,
        "instrutor": {
            "nome": "Ana Santos",
            "email": "ana.santos@emailaleatorio.com",
            "telefone": "(11) 7777-7777"
        }
    },
    {
        "curso": "Inteligência artificial",
        "categoria": "Programação",
        "carga_horaria": "40 horas",
        "concluintes": "",
        "data_inicio": "2022-04-01",
        "data_conclusao": "",
        "descricao": "Curso avançado sobre inteligência artificial com Python",
        "preco": 129.90,
        "instrutor": {
            "nome": "",
            "email": "contato@emailaleatorio.com",
            "telefone": ""
        }
    },
    {
        "curso": "Inglês para negócios",
        "categoria": "Idiomas",
        "carga_horaria": "20 horas",
        "concluintes": 30,
        "data_inicio": "",
        "data_conclusao": "",
        "descricao": "Curso de inglês para negócios",
        "preco": 69.90,
        "instrutor": {
            "nome": "John Smith",
            "email": "",
            "telefone": ""
        }
    },
    {
        "curso": "Introdução à programação",
        "categoria": "Programação",
        "carga_horaria": "20 horas",
        "concluintes": 100,
        "data_inicio": "2022-01-01",
        "data_conclusao": "2022-01-20",
        "descricao": "Curso introdutório à programação com Python",
        "preco": "99.90",
        "instrutor": {
            "nome": "João Silva",
            "email": "joao.silva@emailaleatorio.com",
            "telefone": "(11) 9999-9999"
        }
    }
]
```

In [ ]:
url = 'pandas-dados/cursos_cadastrados.json'

with open(url) as f:
    dados = json.load(f)

In [ ]:
dados_brutos = pd.json_normalize(dados)
dados_brutos

In [ ]:
dados_brutos.info()

In [ ]:
dados_brutos.isnull().sum()

In [ ]:
filtrando = dados_brutos.isna().any(axis=1)

In [ ]:
dados_brutos[filtrando]

In [ ]:
dados_brutos.dropna(inplace=True)

In [ ]:
dados_brutos

In [ ]:
dados_brutos.duplicated()

In [ ]:
dados_brutos.drop_duplicates(inplace=True)

In [ ]:
dados_brutos

In [ ]:
dados_brutos.info()

In [ ]:
dados_brutos[dados_brutos['instrutor.nome'] == "" ]

In [ ]:
dados_brutos[dados_brutos['data_conclusao'] == "" ]

In [ ]:
dados_brutos.replace('', pd.NA, inplace=True)

In [ ]:
dados_brutos

In [ ]:
dados_brutos.dropna(subset='data_conclusao',inplace=True)

In [ ]:
dados_brutos

In [ ]:
dados_brutos.info()

In [ ]:
dados_brutos['preco'] = dados_brutos['preco'].astype(float)
dados_brutos['concluintes'] = dados_brutos['concluintes'].astype(int)
dados_brutos[['data_inicio', 'data_conclusao']] = dados_brutos[['data_inicio', 'data_conclusao']].apply(pd.to_datetime)

In [ ]:
dados_brutos.info()

In [ ]:
type(dados_brutos)

### Desafio: identificando fraudes

Você trabalha em um banco como cientista de dados e recebeu o seguinte conjunto de dados para analisar:

```python
import pandas as pd

# criando um DataFrame com 30 transações aleatórias
df = pd.DataFrame({
    'ID da transação': range(1, 31),
    'Valor da transação': [100, 200, 150, 500, 300, 913, 250, 400, 200, 150, 
                           200, 200, 400, 300, 150, 301, 805, 300, 400, 250, 
                           150, 100, 500, 600, 200, 350, 100, 250, 800, 250],
    'Data da transação': pd.date_range(start='2022-01-01', end='2022-01-30', freq='D'),
    'Local da transação': ['São Paulo, Brasil', 'Rio de Janeiro, Brasil', 'Belo Horizonte, Brasil', 'São Paulo, Brasil', 
                           'São Paulo, Brasil', 'Nova Iorque, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'Rio de Janeiro, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Los Angeles, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Miami, EUA', 'São Paulo, Brasil']
})

```

Foi informado que nessa base de dados há possíveis fraudes que podem ser identificadas através dos outliers no “valor da transação”. Basicamente a fraude seria um outlier.

Sua missão é construir uma função que recebe dois parâmetros, a coluna que você quer analisar se tem outlier e o DataFrame em questão. A intenção de fazer isso é aproveitar novamente essa função tanto para outras colunas quanto para outros DataFrames futuramente em outras análises. O método que deve ser implementado para identificação do outlier é do intervalo interquartil.

Como podemos fazer isso?

In [ ]:
import pandas as pd

# criando um DataFrame com 30 transações aleatórias
df = pd.DataFrame({
    'ID da transação': range(1, 31),
    'Valor da transação': [100, 200, 150, 500, 300, 913, 250, 400, 200, 150, 
                           200, 200, 400, 300, 150, 301, 805, 300, 400, 250, 
                           150, 100, 500, 600, 200, 350, 100, 250, 800, 250],
    'Data da transação': pd.date_range(start='2022-01-01', end='2022-01-30', freq='D'),
    'Local da transação': ['São Paulo, Brasil', 'Rio de Janeiro, Brasil', 'Belo Horizonte, Brasil', 'São Paulo, Brasil', 
                           'São Paulo, Brasil', 'Nova Iorque, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'Rio de Janeiro, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Los Angeles, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Miami, EUA', 'São Paulo, Brasil']
})

In [ ]:
df.info()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(x=df['Valor da transação'])

In [ ]:
Q1 = df['Valor da transação'].quantile(.25)
Q3 = df['Valor da transação'].quantile(.75)
IQR = Q3-Q1
limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR

In [ ]:
filtro_outliers = (df['Valor da transação'] < limite_inferior) | (df['Valor da transação'] > limite_superior)
filtro_outliers

In [ ]:
df[filtro_outliers]

In [ ]:
from pandas import DataFrame

In [ ]:
def verifica_fraude(nome_coluna: str, dataframe: DataFrame) -> DataFrame:
    Q1 = df[nome_coluna].quantile(.25)
    Q3 = df[nome_coluna].quantile(.75)
    IQR = Q3-Q1
    limite_inferior = Q1 - 1.5*IQR
    limite_superior = Q3 + 1.5*IQR

    outlier_filtro = (df[nome_coluna] < limite_inferior) | (df[nome_coluna] > limite_superior)

    outlier = dataframe[outlier_filtro]

    return outlier


In [ ]:
print(f"{verifica_fraude('Valor da transação', df)}")